In [106]:
"""
Tarang TO DO

1. FIX GITIGNORE
    Operates locally.
""";

In [1]:
# Importing Webscraping, data managment and visualization libraries.

import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, random

# Web–scraping & cleaning libs.
import requests
from bs4 import BeautifulSoup

# NLP Libs.
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import re

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier,\
AdaBoostClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,\
ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
#
#import xgboost as xgb

[nltk_data] Downloading package punkt to /Users/tarang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tarang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
""" This is how we will start to extract the lyrics from the song webpage.
"""

url = 'https://www.lyrics.com/lyric/31024783/The+Strokes/Heart+In+a+Cage'
req = requests.get(url)

soup = BeautifulSoup(req.text)
lyrics = soup.find("pre",{'id' : 'lyric-body-text'}).text
lyrics

"Well, I don't feel better when I'm fucking around\nAnd I don't write better when I'm stuck in the ground\nSo don't teach me a lesson 'cause I've already learned\nYeah, the sun will be shining and my children will burn\n\nOh, the heart beats in its cage\n\nI don't want what you want, I don't feel what you feel\nSee, I'm stuck in a city but I belong in a field\nYeah, we got left, left, left, left, left, left, left\nNow it's three in the morning and you're eating alone\n\nOh, the heart beats in its cage\n\nAll our friends, they're laughing at us\nAll of those you loved, you mistrust\nHelp me, I'm just not quite myself\nLook around, there's no one else left\n\nI went to the concert and I fought through the crowd\nGuess I got too excited when I thought you were around\nOh, he gets left, left, left, left, left, left, left\nAnd I'm sorry you were thinking I would steal your fire\n\nOh, the heart beats in its cage\nYes, the heart beats in its cage\n\nAnd the heart beats in its cage"

In [109]:
# \n is a line \n\n is a stanza.
#<div class="clearfix"><h3 class="artist-album-label">

## Make df of lyrics and write to csv for initial NLP model dev. ?

In [110]:
lyrics 
#lyrics.to_csv('./capstone/data/test_lyrics.csv')

"Well, I don't feel better when I'm fucking around\nAnd I don't write better when I'm stuck in the ground\nSo don't teach me a lesson 'cause I've already learned\nYeah, the sun will be shining and my children will burn\n\nOh, the heart beats in its cage\n\nI don't want what you want, I don't feel what you feel\nSee, I'm stuck in a city but I belong in a field\nYeah, we got left, left, left, left, left, left, left\nNow it's three in the morning and you're eating alone\n\nOh, the heart beats in its cage\n\nAll our friends, they're laughing at us\nAll of those you loved, you mistrust\nHelp me, I'm just not quite myself\nLook around, there's no one else left\n\nI went to the concert and I fought through the crowd\nGuess I got too excited when I thought you were around\nOh, he gets left, left, left, left, left, left, left\nAnd I'm sorry you were thinking I would steal your fire\n\nOh, the heart beats in its cage\nYes, the heart beats in its cage\n\nAnd the heart beats in its cage"

In [111]:
temp_df = pd.DataFrame(lyrics.split('\n'))
# add stanza number, song name

def stanzas(df):
    counter = 1
    stanzas = []
    for row in df:
        if row != '':
            stanzas.append(counter)
        else:
            stanzas.append(0)
            counter +=1
    return stanzas
            

In [112]:
stanza_cnt = pd.DataFrame(stanzas(temp_df[0]))
stanza_cnt.head(2)
test = temp_df.merge(stanza_cnt, how = 'left', left_on = temp_df.index, right_on = stanza_cnt.index, left_index=True)
test.to_csv('./data/test_song.csv')

In [113]:
test.head()

,key_0,0_x,0_y
0,0,"Well, I don't feel better when I'm fucking around",1
1,1,And I don't write better when I'm stuck in the...,1
2,2,So don't teach me a lesson 'cause I've already...,1
3,3,"Yeah, the sun will be shining and my children ...",1
4,4,,0


In [114]:
test1 = test.copy()
test1 = test1[test1['0_y']!=0]
test1

,key_0,0_x,0_y
0,0,"Well, I don't feel better when I'm fucking around",1
1,1,And I don't write better when I'm stuck in the...,1
2,2,So don't teach me a lesson 'cause I've already...,1
3,3,"Yeah, the sun will be shining and my children ...",1
5,5,"Oh, the heart beats in its cage",2
7,7,"I don't want what you want, I don't feel what ...",3
8,8,"See, I'm stuck in a city but I belong in a field",3
9,9,"Yeah, we got left, left, left, left, left, lef...",3
10,10,Now it's three in the morning and you're eatin...,3
12,12,"Oh, the heart beats in its cage",4


In [ ]:
"""
Est. API querying; append to empty lists for lines?
Remove stopwords/ est stopwords to start?
Where to use randomizer?
Words coming up with null result will be skipped/ignored (i.e. slang)
Can use cvec to det. new song name? Or can use randomizer from top?

"""

In [10]:
"""
SHOULD I USE SUB FUNCTIONS
""";
user_id = open('./data/lyrics_user.txt', mode = 'r').read()
token = open('./data/lyrics_key.txt', mode = 'r').read()

artist = input('Enter artist here : ')
# if not redirected to artist page (instead results of ARTISTS, return message to include articles in name)
url = 'https://www.lyrics.com/artist/'+str(artist.replace(' ', '+'))
req = requests.get(url)
soup = BeautifulSoup(req.text)
tables = soup.find_all('table', {'class': 'tdata'})
songs = []
for table in tables:
    for row in table.find('tbody').find_all('tr'):
        songs.append(row.find('a').text)
songs = pd.DataFrame(set(songs), columns = ['Titles'])
print(songs)
# how to display FULL disc on Web Pge w/0 elipsis?
album = input('Choose a title by index number : ')
song = str(songs.loc[int(album),'Titles'])
#
# Getting album url to pull tracklist and search lyrics for each.
#
url = 'https://www.stands4.com/services/v2/lyrics.php'
params = {'uid' : str(user_id),
         'tokenid' : str(token),
         'term' : str(song),
         'format': 'json'}
req = requests.get(url, params)
req.json()
#
# Log album name and use url from dict to nav to page and pull track list for further querying
#

Enter artist here : the weeknd
                                  Titles
0                     Stargirl Interlude
1                          What You Need
2                               Some Way
3                       The Birds Part 1
4        Wanderlust [Pharrell Remix] [*]
..                                   ...
165                           Wanderlust
166                       Love Me Harder
167                            Earned It
168  House of Balloons/Glass Table Girls
169                           All I Know

[170 rows x 1 columns]
Choose a title by index number : 10


{'result': [{'song': 'The Birds, Pt. 1',
   'song-link': 'https://www.lyrics.com/lyric/28355646/The+Weeknd/The+Birds%2C+Pt.+1',
   'artist': 'The Weeknd',
   'artist-link': 'https://www.lyrics.com/artist/The+Weeknd/2377309',
   'album': 'Trilogy',
   'album-link': 'https://www.lyrics.com/album/2599514/Trilogy'},
  {'song': 'The Birds, Pt. 1',
   'song-link': 'https://www.lyrics.com/lyric/28394856/The+Weeknd/The+Birds%2C+Pt.+1',
   'artist': 'The Weeknd',
   'artist-link': 'https://www.lyrics.com/artist/The+Weeknd/2377309',
   'album': 'Trilogy',
   'album-link': 'https://www.lyrics.com/album/2599514/Trilogy'},
  {'song': 'The Birds, Pt. 1',
   'song-link': 'https://www.lyrics.com/lyric/32099883/The+Weeknd/The+Birds%2C+Pt.+1',
   'artist': 'The Weeknd',
   'artist-link': 'https://www.lyrics.com/artist/The+Weeknd/2377309',
   'album': 'Thursday',
   'album-link': 'https://www.lyrics.com/album/2284627/Thursday'},
  {'song': 'The Birds, Pt. 1',
   'song-link': 'https://www.lyrics.com/lyric

In [11]:
json_results = req.json()

## Code to capture album data for songs with more than 7 titles

In [12]:
##basic loop that goes through the json output/query result.
counter = 0
for counter in range(len(json_results["result"])):
    ##if json_code output == to artist name input: then grab the album name
    if json_results["result"][counter]['artist'].lower() == artist.lower():
        
        ##code that adds all the songs within the album
        url = json_results["result"][counter]['album-link']
        req = requests.get(url)
        soup = BeautifulSoup(req.text)
        tables = soup.find_all('table', {'class': 'tdata'})
        album_songs = []
        for table in tables:
            for row in table.find('tbody').find_all('tr'):
                
                album_songs.append(row.find('a').text)
        if len(album_songs) >= 7:
            
            ##CODE FOR STORING THE ALBUM AND ITS SONGS INTO A DATA FRAME THAT CONTAINS ALBUM, SONG NAME, SONG URL, 
            ##ARTIST GOES HERE
            
            ##This code grabs all the song urls within the album            
            for table in tables:
                link_list = [a['href'] for a in soup.find_all('a', href=True)]
                lyrics_link = []
                for link in link_list:
                    if link.startswith('/lyric/'):
                        lyrics_link.append(link)
                        
            full_link_list = []
            for link in lyrics_link:
                full_link_list.append('https://www.lyrics.com/'+link)
                
            ##Thies creates the dataframe of the 
            album_data = pd.DataFrame({'Artist':json_results["result"][counter]['artist'],
                           'Album':json_results["result"][counter]['album'],
                           'Song Link':full_link_list,
                           'Song Name':album_songs})            
            
                
 
        counter += 1

In [13]:
album_data

,Artist,Album,Song Link,Song Name
0,The Weeknd,Trilogy,https://www.lyrics.com//lyric/28355660/High+fo...,High for This
1,The Weeknd,Trilogy,https://www.lyrics.com//lyric/28355659/What+Yo...,What You Need
2,The Weeknd,Trilogy,https://www.lyrics.com//lyric/28355658/House+o...,House of Balloons/Glass Table Girls
3,The Weeknd,Trilogy,https://www.lyrics.com//lyric/28355657/The+Mor...,The Morning
4,The Weeknd,Trilogy,https://www.lyrics.com//lyric/28355656/Wicked+...,Wicked Games
5,The Weeknd,Trilogy,https://www.lyrics.com//lyric/28355655/The+Par...,The Party & the After Party
6,The Weeknd,Trilogy,https://www.lyrics.com//lyric/28355654/Coming+...,Coming Down
7,The Weeknd,Trilogy,https://www.lyrics.com//lyric/28355653/Loft+Music,Loft Music
8,The Weeknd,Trilogy,https://www.lyrics.com//lyric/28355652/The+Kno...,The Knowing
9,The Weeknd,Trilogy,https://www.lyrics.com//lyric/28355651/Twenty+...,Twenty Eight


## Stanza + Album Info DataFrame

In [60]:
""" Extracting the links from the album_data dataframe
"""
# link_list = []
# for song_link in album_data['Song Link']:
#     print(song_link)
total_song_links = [song_link for song_link in album_data['Song Link']]
total_song_links

['https://www.lyrics.com//lyric/28355660/High+for+This',
 'https://www.lyrics.com//lyric/28355659/What+You+Need',
 'https://www.lyrics.com//lyric/28355658/House+of+Balloons-Glass+Table+Girls',
 'https://www.lyrics.com//lyric/28355657/The+Morning',
 'https://www.lyrics.com//lyric/28355656/Wicked+Games',
 'https://www.lyrics.com//lyric/28355655/The+Party+%26+the+After+Party',
 'https://www.lyrics.com//lyric/28355654/Coming+Down',
 'https://www.lyrics.com//lyric/28355653/Loft+Music',
 'https://www.lyrics.com//lyric/28355652/The+Knowing',
 'https://www.lyrics.com//lyric/28355651/Twenty+Eight',
 'https://www.lyrics.com//lyric/28355650/Lonely+Star',
 'https://www.lyrics.com//lyric/28355649/Life+of+the+Party',
 'https://www.lyrics.com//lyric/28355648/Thursday',
 'https://www.lyrics.com//lyric/28355647/The+Zone',
 'https://www.lyrics.com//lyric/28355646/The+Birds%2C+Pt.+1',
 'https://www.lyrics.com//lyric/28355645/The+Birds%2C+Pt.+2',
 'https://www.lyrics.com//lyric/28355644/Gone',
 'https://w

In [ ]:
def stanzas(df):
    counter = 1
    stanzas = []
    for row in df:
        if row != '':
            stanzas.append(counter)
        else:
            stanzas.append(0)
            counter +=1
    return stanzas

for song in total_song_links:
    url = song
    req = requests.get(url)
    soup = BeautifulSoup(req.text)
    lyrics = soup.find("pre",{'id' : 'lyric-body-text'}).text
    temp_df = pd.DataFrame(lyrics.split('\n'))


In [15]:
""" This is how we will start to extract the lyrics from the song webpage.
"""

url = 'https://www.lyrics.com//lyric/28355648/Thursday'
req = requests.get(url)

soup = BeautifulSoup(req.text)
lyrics = soup.find("pre",{'id' : 'lyric-body-text'}).text
lyrics

"Ooh, ooh\nOoh, ooh\n\nI love her, today\n'Cause it's Wednesday, I won't be late\nNow I need to know\nIf you're feeling when I'm free\n\nNot on Monday, Tuesday, Wednesday, Friday, Saturday\nSunday, but on Thursday, Thursday\nNot on Monday, Tuesday, Wednesday, Friday, Saturday\nSunday, but on Thursday, make sure\nYou are thirsty oh, on Thursday, baby\nGet ready, oh, on Thursday\nWhy you calling? Oh, Thursday\nJust wait on Thursday, baby\nOnly Thursday\n\nOoh, oh\nGirl I try, girl I try\nI've been here for too long\nBaby, don't cry\nWhy are you calling again? (Oh, again)\nBut it's not Thursday, baby, why you calling?\n\nNot on Monday, Tuesday, Wednesday, Friday, Saturday\nSunday, but on Thursday, Thursday\nNot on Monday, Tuesday, Wednesday, Friday, Saturday, Sunday, but on Thursday, make sure\nYou are thirsty oh, on Thursday, baby\nGet ready, oh, on Thursday\nWhy you holding in, oh, on Thursday\nJust wait on Thursday, baby\nOnly Thursday\n\nThursday\nThursday\nGirl, get beautiful\nThursd

In [16]:
""" This is the Stanza DF that ended up being created!
"""
temp_df = pd.DataFrame(lyrics.split('\n'))
# add stanza number, song name

def stanzas(df):
    counter = 1
    stanzas = []
    for row in df:
        if row != '':
            stanzas.append(counter)
        else:
            stanzas.append(0)
            counter +=1
    return stanzas


In [17]:
stanza_cnt = pd.DataFrame(stanzas(temp_df[0]))
stanza_cnt.head(2)
test = temp_df.merge(stanza_cnt, how = 'left', left_on = temp_df.index, right_on = stanza_cnt.index, left_index=True)


In [18]:
test1 = test.copy()
test1 = test1[test1['0_y']!=0]
test1

,key_0,0_x,0_y
0,0,"Ooh, ooh",1
1,1,"Ooh, ooh",1
3,3,"I love her, today",2
4,4,"'Cause it's Wednesday, I won't be late",2
5,5,Now I need to know,2
6,6,If you're feeling when I'm free,2
8,8,"Not on Monday, Tuesday, Wednesday, Friday, Sat...",3
9,9,"Sunday, but on Thursday, Thursday",3
10,10,"Not on Monday, Tuesday, Wednesday, Friday, Sat...",3
11,11,"Sunday, but on Thursday, make sure",3


In [19]:
test1['Song Link'] = url
test1

,key_0,0_x,0_y,Song Link
0,0,"Ooh, ooh",1,https://www.lyrics.com//lyric/28355648/Thursday
1,1,"Ooh, ooh",1,https://www.lyrics.com//lyric/28355648/Thursday
3,3,"I love her, today",2,https://www.lyrics.com//lyric/28355648/Thursday
4,4,"'Cause it's Wednesday, I won't be late",2,https://www.lyrics.com//lyric/28355648/Thursday
5,5,Now I need to know,2,https://www.lyrics.com//lyric/28355648/Thursday
6,6,If you're feeling when I'm free,2,https://www.lyrics.com//lyric/28355648/Thursday
8,8,"Not on Monday, Tuesday, Wednesday, Friday, Sat...",3,https://www.lyrics.com//lyric/28355648/Thursday
9,9,"Sunday, but on Thursday, Thursday",3,https://www.lyrics.com//lyric/28355648/Thursday
10,10,"Not on Monday, Tuesday, Wednesday, Friday, Sat...",3,https://www.lyrics.com//lyric/28355648/Thursday
11,11,"Sunday, but on Thursday, make sure",3,https://www.lyrics.com//lyric/28355648/Thursday


In [20]:
final_dataframe = pd.merge(test1,album_data,on='Song Link')

In [21]:
final_dataframe

,key_0,0_x,0_y,Song Link,Artist,Album,Song Name
0,0,"Ooh, ooh",1,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday
1,1,"Ooh, ooh",1,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday
2,3,"I love her, today",2,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday
3,4,"'Cause it's Wednesday, I won't be late",2,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday
4,5,Now I need to know,2,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday
5,6,If you're feeling when I'm free,2,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday
6,8,"Not on Monday, Tuesday, Wednesday, Friday, Sat...",3,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday
7,9,"Sunday, but on Thursday, Thursday",3,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday
8,10,"Not on Monday, Tuesday, Wednesday, Friday, Sat...",3,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday
9,11,"Sunday, but on Thursday, make sure",3,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday


In [22]:
final_dataframe['Number of Stanzas'] = final_dataframe['0_y'].max() 

In [23]:
final_dataframe.columns

Index(['key_0', '0_x', '0_y', 'Song Link', 'Artist', 'Album', 'Song Name',
       'Number of Stanzas'],
      dtype='object')

In [24]:
final_dataframe2 = final_dataframe.copy()

In [25]:
final_dataframe2.columns = [column_name.replace(" ","_") for column_name in final_dataframe2.columns]

In [26]:
final_dataframe2

,key_0,0_x,0_y,Song_Link,Artist,Album,Song_Name,Number_of_Stanzas
0,0,"Ooh, ooh",1,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7
1,1,"Ooh, ooh",1,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7
2,3,"I love her, today",2,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7
3,4,"'Cause it's Wednesday, I won't be late",2,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7
4,5,Now I need to know,2,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7
5,6,If you're feeling when I'm free,2,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7
6,8,"Not on Monday, Tuesday, Wednesday, Friday, Sat...",3,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7
7,9,"Sunday, but on Thursday, Thursday",3,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7
8,10,"Not on Monday, Tuesday, Wednesday, Friday, Sat...",3,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7
9,11,"Sunday, but on Thursday, make sure",3,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7


In [34]:
final_dataframe2 = final_dataframe2.drop(['Number_of_Stanzas'], axis=1)

In [44]:
final_dataframe2.columns = ['Line_Num','Stanza','Stanza_Num','Song_Link', 'Artist', 'Album', 'Song_Name']

In [53]:
final_dataframe2[final_dataframe2.Song_Name == 'Thursday'].Stanza_Num.max()

7

In [54]:
final_dataframe2['Total_Stanzas'] = final_dataframe2[final_dataframe2.Song_Name == 'Thursday'].Stanza_Num.max()
final_dataframe2

,Line_Num,Stanza,Stanza_Num,Song_Link,Artist,Album,Song_Name,Total_Stanzas
0,0,"Ooh, ooh",1,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7
1,1,"Ooh, ooh",1,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7
2,3,"I love her, today",2,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7
3,4,"'Cause it's Wednesday, I won't be late",2,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7
4,5,Now I need to know,2,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7
5,6,If you're feeling when I'm free,2,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7
6,8,"Not on Monday, Tuesday, Wednesday, Friday, Sat...",3,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7
7,9,"Sunday, but on Thursday, Thursday",3,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7
8,10,"Not on Monday, Tuesday, Wednesday, Friday, Sat...",3,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7
9,11,"Sunday, but on Thursday, make sure",3,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7


In [55]:
final_dataframe2['Total_Lines'] = final_dataframe2[final_dataframe2.Song_Name == 'Thursday'].Line_Num.max()
final_dataframe2

,Line_Num,Stanza,Stanza_Num,Song_Link,Artist,Album,Song_Name,Total_Stanzas,Total_Lines
0,0,"Ooh, ooh",1,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7,52
1,1,"Ooh, ooh",1,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7,52
2,3,"I love her, today",2,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7,52
3,4,"'Cause it's Wednesday, I won't be late",2,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7,52
4,5,Now I need to know,2,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7,52
5,6,If you're feeling when I'm free,2,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7,52
6,8,"Not on Monday, Tuesday, Wednesday, Friday, Sat...",3,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7,52
7,9,"Sunday, but on Thursday, Thursday",3,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7,52
8,10,"Not on Monday, Tuesday, Wednesday, Friday, Sat...",3,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7,52
9,11,"Sunday, but on Thursday, make sure",3,https://www.lyrics.com//lyric/28355648/Thursday,The Weeknd,Trilogy,Thursday,7,52


In [67]:
final_dataframe2['Stanza'].value_counts()

Thursday                                                                                   12
Not on Monday, Tuesday, Wednesday, Friday, Saturday                                         3
Yeah-eh, woah, woah, oh oh                                                                  2
Ooh, ooh                                                                                    2
Girl, get beautiful                                                                         2
Get ready, oh, on Thursday                                                                  2
Sunday, but on Thursday, Thursday                                                           2
You are thirsty oh, on Thursday, baby                                                       2
Just wait on Thursday, baby                                                                 2
Only Thursday                                                                               2
Why you holding in, oh, on Thursday                         

In [66]:
##This is the code to capture the chorus
final_dataframe2['Stanza'].value_counts().index[0]

'Thursday'

## End of Tarang Work

In [43]:
artist

'the strokes'

In [47]:
"""
INITIAL ARTIST PAGE QUERY
"""


res = req.json()
df = res['result']
df
new_df = {}
for i in df:
    for key, value in i.items():
        if value.lower() == artist.lower():
            new_df.update(i)
            break
print(new_df)

"""
How to deal w/ duplicative results on an EP?
    Attribution for albums 'studio', etc.?
""";

{'song': 'Barely Legal', 'song-link': 'https://www.lyrics.com/lyric/35821913/The+Strokes/Barely+Legal', 'artist': 'The Strokes', 'artist-link': 'https://www.lyrics.com/artist/The+Strokes/478260', 'album': 'Is This It [180 Gram Vinyl]', 'album-link': 'https://www.lyrics.com/album/2367673/Is This It [180 Gram Vinyl]'}


In [ ]:
"""
Return tracklist from album page
go to track page & extract lyrics for each
dev DF & attribution
model
pos. sentiment analysis from meta returned from mw dic.?
option for 'clean' versions?


MAYBE two options: new song/clean song??
"""

In [46]:
album = input('Choose a title by index number : ')
str(songs.loc[int(album),'Titles'])

Choose a title by index number : 6


'New York City Cops'

In [41]:
# only works for scalars
#songs.at(5, 'Titles')

TypeError: '_AtIndexer' object is not callable

In [26]:
token = pd.read_csv('./data/lyrics_key.txt', )
print(token)

Empty DataFrame
Columns: [5fGYJwlI810Y5kTs]
Index: []


In [27]:
""" Use this to start to document attributes of music related to song req.
We can store information about the album, artist and song name while navigating to the album page.
From album page we can then loop through song tags and request to their respective pages. Lyric Query is above.
"""

# FN passing through index # of desired song. will work off assumption that fav song will be on album w/ sim tracks

user_id = pd.read_csv('./data/lyrics_user.txt')
token = pd.read_csv('./data/lyrics_key.txt')
song = 'Selfless'

url = 'https://www.stands4.com/services/v2/lyrics.php'
params = {'uid' : str(user_id),
         'tokenid' : str(token),
         'term' : str(song),
         'format': 'json'}
req = requests.get(url, params)
req.json()

#### Should be sub fn using specified artist to refine results to single dict. w/ key:val pairs to document & nav.

{'error': 'Invalid Token'}

----
----
---

### MW API

---

In [2]:
thesarus_key = open('./data/mw_key.txt', mode = 'r').read()
words = ['nice']
params = {
    'key' : thesarus_key
         }
for word in words:
    mw_url = 'https://www.dictionaryapi.com/api/v3/references/thesaurus/json/'
    mw_url += word
    req = requests.get(mw_url, params)
    res = req.json()
    print(res)
    
    #     df = res['meta']
#     df
#     new_df = {}
#     for i in df:
#         for key, value in i.items():
#             if value.lower() == artist.lower():
#                 new_df.update(i)
#                 break
#     print(new_df)

NameError: name 'requests' is not defined

In [65]:
res[0].keys()

dict_keys(['meta', 'hwi', 'fl', 'def', 'shortdef'])

In [66]:
res[0]['def']

[{'sseq': [[['sense',
     {'sn': '1',
      'dt': [['text',
        'following the established traditions of refined society and good taste '],
       ['vis', [{'t': 'had {it}nice{/it} manners'}]]],
      'syn_list': [[{'wd': 'befitting'},
        {'wd': 'correct'},
        {'wd': 'de rigueur'},
        {'wd': 'decent'},
        {'wd': 'decorous'},
        {'wd': 'genteel'},
        {'wd': 'polite'},
        {'wd': 'proper'},
        {'wd': 'respectable'},
        {'wd': 'seemly'}]],
      'rel_list': [[{'wd': 'acceptable'},
        {'wd': 'adequate'},
        {'wd': 'satisfactory'},
        {'wd': 'tolerable'}],
       [{'wd': 'dress'}, {'wd': 'dressy'}, {'wd': 'formal'}],
       [{'wd': 'dignified'}, {'wd': 'elegant'}, {'wd': 'gracious'}],
       [{'wd': 'priggish'}, {'wd': 'prim'}, {'wd': 'stiff'}, {'wd': 'stuffy'}],
       [{'wd': 'apt'}, {'wd': 'material'}, {'wd': 'relevant'}],
       [{'wd': 'compatible'}, {'wd': 'congenial'}, {'wd': 'harmonious'}],
       [{'wd': 'allowed'},
  

In [1]:
# https://dictionaryapi.com/products/json#sec-2.labels
#res[0]['fl'] --> word type

syns = []
for i in res[0]['def']:
    for n in i['sseq']:
        for x in n:
            
            for y in x:
                if type(y) == dict:
                    #print(y.keys())
                    for i in y['syn_list']:
                        for j['wd'] in i:
                            syns.append(j['wd'])

NameError: name 'res' is not defined

In [117]:
syns = pd.DataFrame(syns)['wd']
syns.head()

0     befitting
1       correct
2    de rigueur
3        decent
4      decorous
Name: wd, dtype: object

In [122]:
a = 'This is a line in a song using the word nice'
a.replace('nice', random.choice(list(syns)))

'This is a line in a song using the word tasty'

In [69]:
y['syn_list']

[[{'wd': 'ceremonious'},
  {'wd': 'correct'},
  {'wd': 'decorous'},
  {'wd': 'formal'},
  {'wd': 'proper'},
  {'wd': 'punctilious'},
  {'wd': 'starchy'},
  {'wd': 'stiff'},
  {'wd': 'stiff-necked'},
  {'wd': 'stilted'}]]

In [51]:
test = ['a','b','c']
for i in test:
    print(test.index(i))

0
1
2


In [29]:
get_meta(example)

https://www.dictionaryapi.com/api/v3/references/thesaurus/json/umpire?key=6d207136-9763-48c2-b218-6369e35e2036
https://www.dictionaryapi.com/api/v3/references/thesaurus/json/pussy?key=6d207136-9763-48c2-b218-6369e35e2036


In [19]:
req = requests.get(mw_url, params)

In [20]:
print(req.url)

https://www.dictionaryapi.com/api/v3/references/thesaurus/json/?key=6d207136-9763-48c2-b218-6369e35e2036


In [21]:
req.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)